In [1]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [3]:
!pip install liac-arff

  Preparing metadata (setup.py) ... done
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11717 sha256=4a4ceb9396ad38bbb4b1c9cfce6180cf110c189761f6822ac673b8ac05d412d8
  Stored in directory: /root/.cache/pip/wheels/5d/2a/9c/3895d9617f8f49a0883ba686326d598e78a1c2f54fe3cae86d
Successfully built liac-arff


In [5]:
!pip install pandas scikit-learn


In [37]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, KBinsDiscretizer, MinMaxScaler
import arff

# Cargar el dataset (reemplaza 'diabetes.csv' con la ruta a tu archivo CSV)
df = pd.read_csv('/content/Drive/MyDrive/datos/diabetes.csv')

# Mostrar las primeras filas del dataset original
print("Primeras filas del dataset original:")
print(df.head())


Primeras filas del dataset original:
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [38]:
# Identificar columnas con valores faltantes representados por 0
columns_with_zero = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Reemplazar 0 por NaN
df[columns_with_zero] = df[columns_with_zero].replace(0, pd.NA)

# Mostrar la cantidad de valores faltantes por columna
print("\nValores faltantes por columna:")
print(df.isna().sum())

# Imputar valores faltantes con la media de cada columna
df[columns_with_zero] = df[columns_with_zero].fillna(df[columns_with_zero].mean())

# Verificar que no haya más valores faltantes
print("\nValores faltantes después de la imputación:")
print(df.isna().sum())



Valores faltantes por columna:
Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

Valores faltantes después de la imputación:
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


<ipython-input-38-3211d1026cba>:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[columns_with_zero] = df[columns_with_zero].fillna(df[columns_with_zero].mean())


In [ ]:
DISCRETIZACION

In [39]:
from sklearn.preprocessing import KBinsDiscretizer

# Discretización de 'Age' en 4 bins
kbd_age = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='uniform')
df['Age Discretized'] = kbd_age.fit_transform(df[['Age']]).astype(int)

# Mapeo de los bins a etiquetas
age_labels = ['<30', '30-40', '40-50', '50+']
df['Age Discretized'] = df['Age Discretized'].map({i: label for i, label in enumerate(age_labels)})

# Discretización de 'Glucose' en 3 bins
kbd_glucose = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
df['Glucose Discretized'] = kbd_glucose.fit_transform(df[['Glucose']]).astype(int)

# Mapeo de los bins a etiquetas
glucose_labels = ['Normal', 'High', 'Very High']
df['Glucose Discretized'] = df['Glucose Discretized'].map({i: label for i, label in enumerate(glucose_labels)})

# Mostrar las nuevas columnas discretizadas
print("\nDataset después de la discretización:")
print(df[['Age', 'Age Discretized', 'Glucose', 'Glucose Discretized']].head())



Dataset después de la discretización:
   Age Age Discretized  Glucose Glucose Discretized
0   50           30-40    148.0           Very High
1   31             <30     85.0              Normal
2   32             <30    183.0           Very High
3   21             <30     89.0              Normal
4   33             <30    137.0           Very High


LABELENCODER

In [40]:
from sklearn.preprocessing import LabelEncoder

# Inicializar el codificador
le_age = LabelEncoder()
le_glucose = LabelEncoder()

# Aplicar Label Encoding a 'Age Discretized'
df['Age Encoded'] = le_age.fit_transform(df['Age Discretized'])

# Aplicar Label Encoding a 'Glucose Discretized'
df['Glucose Encoded'] = le_glucose.fit_transform(df['Glucose Discretized'])

# Mostrar las columnas codificadas
print("\nDataset después de Label Encoding:")
print(df[['Age Discretized', 'Age Encoded', 'Glucose Discretized', 'Glucose Encoded']].head())



Dataset después de Label Encoding:
  Age Discretized  Age Encoded Glucose Discretized  Glucose Encoded
0           30-40            0           Very High                2
1             <30            3              Normal                1
2             <30            3           Very High                2
3             <30            3              Normal                1
4             <30            3           Very High                2


ONEHOTENCODER

In [41]:
# One-Hot Encoding para 'Age Discretized'
df = pd.get_dummies(df, columns=['Age Discretized'], prefix='Age')

# One-Hot Encoding para 'Glucose Discretized'
df = pd.get_dummies(df, columns=['Glucose Discretized'], prefix='Glucose')

# Convertir columnas One-Hot Encoding a 0 y 1 (si están en boolean)
one_hot_columns = [col for col in df.columns if col.startswith('Age_') or col.startswith('Glucose_')]
df[one_hot_columns] = df[one_hot_columns].astype(int)

# Mostrar las nuevas columnas después de One-Hot Encoding
print("\nDataset después de One-Hot Encoding:")
print(df[one_hot_columns].head())



Dataset después de One-Hot Encoding:
   Age_30-40  Age_40-50  Age_50+  Age_<30  Glucose_High  Glucose_Normal  \
0          1          0        0        0             0               0   
1          0          0        0        1             0               1   
2          0          0        0        1             0               0   
3          0          0        0        1             0               1   
4          0          0        0        1             0               0   

   Glucose_Very High  
0                  1  
1                  0  
2                  1  
3                  0  
4                  1  


NORMALIZACION

In [42]:
from sklearn.preprocessing import MinMaxScaler

# Inicializar el escalador
scaler = MinMaxScaler()

# Columnas a normalizar
columns_to_normalize = ['Glucose', 'Insulin', 'BMI', 'Age']

# Aplicar la normalización
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# Mostrar las columnas normalizadas
print("\nDataset después de la Normalización:")
print(df[['Glucose', 'Insulin', 'BMI', 'Age']].head())



Dataset después de la Normalización:
    Glucose   Insulin       BMI       Age
0  0.670968  0.170130  0.314928  0.483333
1  0.264516  0.170130  0.171779  0.166667
2  0.896774  0.170130  0.104294  0.183333
3  0.290323  0.096154  0.202454  0.000000
4  0.600000  0.185096  0.509202  0.200000
